In [163]:
import numpy as np
import skimage as ski
from skimage import io
import matplotlib.pyplot as plt
import matplotlib.image as mpltimg
import os
import cv2
from PIL import Image
import keras as krs
import tensorflow as tf
from skimage.morphology import erosion, disk
from skimage.filters import threshold_triangle

In [164]:
#trainin stuffs
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.utils import conv_utils

In [165]:
dataset = []
labels = []
# importing the dataset first for train/test 
# labels are binary (1, 0)
# 1 stands for infected
# 0 stands for control

In [166]:
# here goes the infected batch of data
infected_imgs = os.listdir('E:\DEMO\data_set\infected')
for i, image_name in enumerate(infected_imgs):
    img = cv2.imread('E:\DEMO\data_set\infected\\' +  image_name)
    img = Image.fromarray(img, mode='RGB')
    img = img.resize((100, 100))
    dataset.append(np.array(img))
    labels.append(1)

In [167]:
# here goes the control batch of data
control_imgs = os.listdir('E:\DEMO\data_set\control')
for i, image_name in enumerate(control_imgs):
    img = cv2.imread('E:\DEMO\data_set\control\\' + image_name)
    img = Image.fromarray(img, mode='RGB')
    img = img.resize((100, 100))
    dataset.append(np.array(img))
    labels.append(0)

In [168]:
# converting dataset and labels from lists to np.ndarray() for the upcoming work
dataset = np.array(dataset)
labels = np.array(labels)

In [169]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(dataset, labels, test_size = 0.20, random_state = 0)

In [170]:
x_train = tf.keras.utils.normalize(x_train, axis=1)
x_test = tf.keras.utils.normalize(x_test, axis=1)

In [171]:
#creatin za model
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(100, 100, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, (3, 3), kernel_initializer='he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), kernel_initializer='he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))
#we are performing a binary classification
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
print(model.summary())


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 98, 98, 32)        896       
_________________________________________________________________
activation_6 (Activation)    (None, 98, 98, 32)        0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 49, 49, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 47, 47, 32)        9248      
_________________________________________________________________
activation_7 (Activation)    (None, 47, 47, 32)        0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 23, 23, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 21, 21, 64)       

In [172]:
history = model.fit(x_train, y_train, batch_size = 64, verbose = 1, epochs = 10, validation_data = (x_test, y_test), shuffle = False)
model.save('covid_lungs_infection_model.h5')

Epoch 1/10
1/1 [==============================] - 6s 6s/step - loss: 0.6939 - accuracy: 0.5000 - val_loss: 0.6971 - val_accuracy: 0.5000
Epoch 2/10
1/1 [==============================] - 1s 784ms/step - loss: 0.7621 - accuracy: 0.3125 - val_loss: 0.6954 - val_accuracy: 0.5000
Epoch 3/10
1/1 [==============================] - 1s 725ms/step - loss: 0.6940 - accuracy: 0.5000 - val_loss: 0.6925 - val_accuracy: 0.5000
Epoch 4/10
1/1 [==============================] - 1s 566ms/step - loss: 0.7213 - accuracy: 0.1875 - val_loss: 0.6921 - val_accuracy: 0.5000
Epoch 5/10
1/1 [==============================] - 0s 468ms/step - loss: 0.6871 - accuracy: 0.6250 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 6/10
1/1 [==============================] - 0s 370ms/step - loss: 0.6864 - accuracy: 0.6875 - val_loss: 0.6922 - val_accuracy: 0.5000
Epoch 7/10
1/1 [==============================] - 0s 346ms/step - loss: 0.7004 - accuracy: 0.3750 - val_loss: 0.6920 - val_accuracy: 0.5000
Epoch 8/10
1/1 [=======